<a href="https://colab.research.google.com/github/hmh10098/d2l/blob/main/5_2_Parameter_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

In [41]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(2, 4)

In [42]:
net(X)

tensor([[0.2009],
        [0.3172]], grad_fn=<AddmmBackward>)

In [8]:
net[0].state_dict()

OrderedDict([('weight', tensor([[-0.0082, -0.4899, -0.4838, -0.0246],
                      [-0.2353,  0.0798, -0.0892, -0.2580],
                      [ 0.3217,  0.2536,  0.2998, -0.1407],
                      [-0.2462, -0.1438,  0.2770, -0.1385],
                      [ 0.3223, -0.4210, -0.3890, -0.1851],
                      [ 0.1834, -0.2970,  0.1929,  0.4921],
                      [-0.0509, -0.1727, -0.4528, -0.4975],
                      [-0.1397, -0.2732,  0.0072,  0.3466]])),
             ('bias',
              tensor([ 0.2804, -0.1825, -0.4707,  0.0516,  0.3236, -0.3558, -0.0218,  0.2846]))])

In [9]:
net[1].state_dict()

OrderedDict()

In [10]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.2968, -0.2430,  0.2139,  0.1405,  0.3342,  0.0851, -0.0116,  0.2667]])),
             ('bias', tensor([-0.0168]))])

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0168], requires_grad=True)
tensor([-0.0168])


In [14]:
net[2].weight.grad == None

True

In [21]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [22]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [23]:
net.state_dict()['2.bias'].data

tensor([-0.0168])

In [24]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0742],
        [0.0743]], grad_fn=<AddmmBackward>)

In [25]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [30]:
rgnet[0][1][0].bias.data
print(*[(name, param.shape) for name, param in rgnet.named_parameters()])
## two way to get data, compare them 
print(rgnet.state_dict()['0.block 1.0.bias'].data == rgnet[0][1][0].bias.data)

('0.block 0.0.weight', torch.Size([8, 4])) ('0.block 0.0.bias', torch.Size([8])) ('0.block 0.2.weight', torch.Size([4, 8])) ('0.block 0.2.bias', torch.Size([4])) ('0.block 1.0.weight', torch.Size([8, 4])) ('0.block 1.0.bias', torch.Size([8])) ('0.block 1.2.weight', torch.Size([4, 8])) ('0.block 1.2.bias', torch.Size([4])) ('0.block 2.0.weight', torch.Size([8, 4])) ('0.block 2.0.bias', torch.Size([8])) ('0.block 2.2.weight', torch.Size([4, 8])) ('0.block 2.2.bias', torch.Size([4])) ('0.block 3.0.weight', torch.Size([8, 4])) ('0.block 3.0.bias', torch.Size([8])) ('0.block 3.2.weight', torch.Size([4, 8])) ('0.block 3.2.bias', torch.Size([4])) ('1.weight', torch.Size([1, 4])) ('1.bias', torch.Size([1]))
tensor([True, True, True, True, True, True, True, True])


In [43]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)

net.apply(init_normal)
net(X)

tensor([[-6.4436e-05],
        [-1.8399e-04]], grad_fn=<AddmmBackward>)

In [48]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 1)
    nn.init.zeros_(m.bias)
net.apply(init_normal)
net(X)

tensor([[ 5.1558],
        [18.6823]], grad_fn=<AddmmBackward>)

In [50]:
net[2].weight

Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)

In [51]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5883, -0.5034, -0.3582, -0.3457])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [53]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-7.3735, -5.1310,  0.0000, -8.1037],
        [ 0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000, -6.3452],
        [-8.6255,  0.0000,  7.0919, -5.4661],
        [-5.7562,  9.4430, -9.1966,  0.0000],
        [-0.0000, -7.0008,  5.6787,  5.1963],
        [ 9.5823,  9.2447, -7.3514, -5.6011],
        [ 9.3772,  0.0000, -8.4972, -6.2785]], requires_grad=True)

In [54]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -4.1310,  1.0000, -7.1037])

In [56]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))

In [57]:
net(X)


tensor([[0.3295],
        [0.3503]], grad_fn=<AddmmBackward>)

In [58]:
print(net[2].weight.data == net[4].weight.data)

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])


In [60]:
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
